# Exploratory Data Analysis (EDA)

In [3]:
# Import necessary libraries
import pandas as pd

# Load the datasets
customer_df = pd.read_csv('C:\\Users\\Admin\\Downloads\\Customers.csv')  
product_df = pd.read_csv('C:\\Users\\Admin\\Downloads\\Products.csv')    
transaction_df = pd.read_csv('C:\\Users\\Admin\\Downloads\\Transactions.csv')  

In [5]:
# Show the first few rows of each dataset to understand the structure
print("Customer Data:")
print(customer_df.head())

Customer Data:
  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15


In [6]:
# Show the first few rows of each dataset to understand the structure
print("\nProduct Data:")
print(product_df.head())


Product Data:
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31


In [7]:
# Show the first few rows of each dataset to understand the structure
print("\nTransaction Data:")
print(transaction_df.head())


Transaction Data:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price  
0      300.68  300.68  
1      300.68  300.68  
2      300.68  300.68  
3      601.36  300.68  
4      902.04  300.68  


In [8]:
# Merge the datasets (assuming 'CustomerID' and 'ProductID' are the common identifiers)
merged_df = pd.merge(transaction_df, customer_df, on='CustomerID', how='inner')
merged_df = pd.merge(merged_df, product_df, on='ProductID', how='inner')


In [9]:
# Sample Analysis 1: Total sales by region (group by region)
total_sales_by_region = merged_df.groupby('Region')['TotalValue'].sum().reset_index()
print("\nTotal Sales by Region:")
print(total_sales_by_region)



Total Sales by Region:
          Region  TotalValue
0           Asia   152074.97
1         Europe   166254.63
2  North America   152313.40
3  South America   219352.56


In [22]:
# Sample Analysis 2: Top-selling products by quantity sold
top_selling_products = merged_df.groupby('ProductName')['Quantity'].sum().reset_index()
top_selling_products = top_selling_products.sort_values(by='Quantity', ascending=False)
print("\nTop-selling Products:")
top_selling_products


Top-selling Products:


,ProductName,Quantity
9,ActiveWear Smartwatch,100
47,SoundWave Headphones,97
36,HomeSense Desk Lamp,81
6,ActiveWear Rug,79
45,SoundWave Cookbook,78
...,...,...
37,HomeSense Headphones,18
50,SoundWave Laptop,16
48,SoundWave Jacket,16
22,BookWorld Wall Art,15


In [11]:
# Sample Analysis 3: Average transaction value by customer region
avg_transaction_value_by_region = merged_df.groupby('Region')['TotalValue'].mean().reset_index()
print("\nAverage Transaction Value by Region:")
print(avg_transaction_value_by_region)


Average Transaction Value by Region:
          Region  TotalValue
0           Asia  697.591606
1         Europe  710.489872
2  North America  624.235246
3  South America  721.554474


In [12]:
# Sample Analysis 4: Most recent transactions by customer
merged_df['TransactionDate'] = pd.to_datetime(merged_df['TransactionDate'])  # Convert to datetime format
most_recent_transactions = merged_df.sort_values('TransactionDate', ascending=False).groupby('CustomerID').first().reset_index()
print("\nMost Recent Transactions by Customer:")
print(most_recent_transactions)


Most Recent Transactions by Customer:
    CustomerID TransactionID ProductID     TransactionDate  Quantity  \
0        C0001        T00436      P029 2024-11-02 17:04:16         3   
1        C0002        T00675      P071 2024-12-03 01:41:41         2   
2        C0003        T00648      P002 2024-08-24 18:54:04         4   
3        C0004        T00379      P077 2024-12-23 14:13:52         4   
4        C0005        T00789      P012 2024-11-04 00:30:22         2   
..         ...           ...       ...                 ...       ...   
194      C0196        T00575      P079 2024-12-15 03:43:35         4   
195      C0197        T00951      P027 2024-12-27 18:20:31         2   
196      C0198        T00583      P073 2024-10-04 18:31:12         1   
197      C0199        T00963      P008 2024-10-26 00:01:58         2   
198      C0200        T00518      P034 2024-12-11 03:05:50         4   

     TotalValue  Price_x        CustomerName         Region  SignupDate  \
0       1300.92   433

In [13]:
# Save the merged data for further analysis
merged_df.to_csv('merged_data.csv', index=False)

In [14]:
# Customer Segmentation Based on Total Spending

# Group by CustomerID and calculate total spending per customer
customer_spending = merged_df.groupby('CustomerID')['TotalValue'].sum().reset_index()

In [15]:
# Create bins for segmentation (Low, Medium, High spenders)
# Define the spending ranges for Low, Medium, High (You can adjust these based on your dataset's distribution)
bins = [0, 500, 1000, customer_spending['TotalValue'].max()]
labels = ['Low', 'Medium', 'High']

In [16]:
# Assign a segment to each customer based on their total spending
customer_spending['SpendingCategory'] = pd.cut(customer_spending['TotalValue'], bins=bins, labels=labels, include_lowest=True)


In [17]:
# Merge the spending categories back into the main DataFrame
merged_with_spending = pd.merge(merged_df, customer_spending[['CustomerID', 'SpendingCategory']], on='CustomerID', how='left')


In [18]:
# Show the total sales by spending category
sales_by_spending_category = merged_with_spending.groupby('SpendingCategory')['TotalValue'].sum().reset_index()


In [19]:
# Show the number of customers in each segment
customer_count_by_spending_category = customer_spending['SpendingCategory'].value_counts().reset_index()
customer_count_by_spending_category.columns = ['SpendingCategory', 'CustomerCount']


In [20]:
# Display the results
print("\nTotal Sales by Spending Category:")
print(sales_by_spending_category)



Total Sales by Spending Category:
  SpendingCategory  TotalValue
0              Low     1291.50
1           Medium     9771.09
2             High   678932.97


In [21]:
print("\nCustomer Count by Spending Category:")
print(customer_count_by_spending_category)



Customer Count by Spending Category:
  SpendingCategory  CustomerCount
0             High            181
1           Medium             12
2              Low              6
